In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
data=pd.read_csv("/kaggle/input/fertilizer-prediction/Fertilizer Prediction.csv")
data.head()

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/fertilizer-prediction/Fertilizer Prediction.csv'

## Dataset Informations

In [ ]:
data.describe()

### Data Preprocessing

#### Finding the length of the Dataset

In [ ]:
data.shape

#### Finding the missing values

In [ ]:
data.isnull().sum()

**No Missing values detected**

### Visualizing the Dataset

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(data=data,x='Fertilizer Name')

### Removing Categorical Variable from the Dataset which is soil type and crop type

In [ ]:
data['Soil Type'].value_counts()

In [ ]:
data['Crop Type'].value_counts()

### As Soil Type and Crop Type are Categorical Variable mapping them to a numerical variable for good model accuracy

In [ ]:
soil_dict={
    'Loamy':1,
    'Sandy':2,
    'Clayey':3,
    'Black':4,
    'Red':5
}

crop_dict={
    'Sugarcane':1,
    'Cotton':2,
    'Millets':3,
    'Paddy':4,
    'Pulses':5,
    'Wheat':6,
    'Tobacco':7,
    'Barley':8,
    'Oil seeds':9,
    'Ground Nuts':10,
    'Maize':11
    
}

In [ ]:
data['Soil_Num']=data['Soil Type'].map(soil_dict)
data['Crop_Num']=data['Crop Type'].map(crop_dict)

In [ ]:
data=data.drop(['Soil Type','Crop Type'],axis=1)
data.head()

#### Splitting the Dataset into X and Y 

In [ ]:
X=data.drop(['Fertilizer Name'],axis=1)
Y=data['Fertilizer Name']

### Splitting Dataset into Train and Test for checking the Accuracy

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

#### Model Building 

**Importing all the Classifier Algorithms**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

# create instances of all models
models = {
    'Logistic Regression': LogisticRegression(),
    'Naive Bayes': GaussianNB(),
    'Support Vector Machine': SVC(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Bagging': BaggingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'Extra Trees': ExtraTreeClassifier(),
}

for name,md in models.items():
    md.fit(X_train,Y_train)
    ypred=md.predict(X_test)
    
    print(f"the Accuracy of {name} is ",accuracy_score(Y_test,ypred))
    


### Let's Take Decision Tree Classifier for our model building

In [ ]:
classifier=DecisionTreeClassifier()
classifier.fit(X_train,Y_train)
ypred=classifier.predict(X_test)
accuracy_score(Y_test,ypred)

In [ ]:
def recommendation(Temparature,Humidity,Moisture,Nitrogen,Potassium,Phosphorous,Soil_Num,Crop_Num):
    features = np.array([[Temparature,Humidity,Moisture,Nitrogen,Potassium,Phosphorous,Soil_Num,Crop_Num]])
    prediction = classifier.predict(features).reshape(1,-1)
    
    return prediction[0] 

In [ ]:
Temparature=2
Humidity=59
Moisture=3
Nitrogen=12
Potassium=0
Phosphorous=3
Soil_Num=2
Crop_Num=11
predict=recommendation(Temparature,Humidity,Moisture,Nitrogen,Potassium,Phosphorous,Soil_Num,Crop_Num)
predict[0]

In [ ]:
### Create a Pickle file using serialization 
import pickle
pickle_out = open("Fertclassifier.pkl","wb")
pickle.dump(classifier, pickle_out)
pickle_out.close()

In [3]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Load dataset
data = pd.read_csv("/kaggle/input/fertilizer-prediction/Fertilizer Prediction.csv")

# Mapping
soil_dict = {
    'Loamy': 1,
    'Sandy': 2,
    'Clayey': 3,
    'Black': 4,
    'Red': 5
}

crop_dict = {
    'Sugarcane': 1,
    'Cotton': 2,
    'Millets': 3,
    'Paddy': 4,
    'Pulses': 5,
    'Wheat': 6,
    'Tobacco': 7,
    'Barley': 8,
    'Oil seeds': 9,
    'Ground Nuts': 10,
    'Maize': 11
}

# Replace categorical with numeric
data['Soil_Num'] = data['Soil Type'].map(soil_dict)
data['Crop_Num'] = data['Crop Type'].map(crop_dict)

# Drop old columns
data = data.drop(['Soil Type', 'Crop Type'], axis=1)

# Features & labels
X = data.drop(['Fertilizer Name'], axis=1)
Y = data['Fertilizer Name']

# Train/test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Model training
classifier = DecisionTreeClassifier()
classifier.fit(X_train, Y_train)

# Accuracy check
ypred = classifier.predict(X_test)
print("Decision Tree Accuracy:", accuracy_score(Y_test, ypred))

# Save model + mappings in one pickle file
model_data = {
    "model": classifier,
    "soil_dict": soil_dict,
    "crop_dict": crop_dict
}

with open("Fertclassifier.pkl", "wb") as f:
    pickle.dump(model_data, f)

print("✅ Fertclassifier.pkl created successfully!")


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/fertilizer-prediction/Fertilizer Prediction.csv'

In [4]:
# 1. Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import pickle

# 2. Load dataset
data = pd.read_csv("Fertilizer Prediction.csv")  # make sure file is in same folder
print("Dataset Loaded Successfully!")
display(data.head())

# 3. Features and Target
X = data[['Temperature', 'Humidity', 'Moisture', 'Soil Type', 'Crop Type',
          'Nitrogen', 'Potassium', 'Phosphorc']]
y = data['Fertilizer Name']

# 4. One-hot encode categorical columns (Soil Type, Crop Type)
X = pd.get_dummies(X, columns=['Soil Type', 'Crop Type'])

# 5. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 6. Train model
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# 7. Save the model as .pkl
with open("fertilizer_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("✅ fertilizer_model.pkl created successfully!")

# 8. Test the model with a sample input
sample = pd.DataFrame({
    'Temperature': [30],
    'Humidity': [60],
    'Moisture': [40],
    'Soil Type': ['Loamy'],
    'Crop Type': ['Sugarcane'],
    'Nitrogen': [20],
    'Potassium': [10],
    'Phosphorc': [15]
})

# Match the columns to the training data
sample = pd.get_dummies(sample)
sample = sample.reindex(columns=X.columns, fill_value=0)

prediction = model.predict(sample)
print("Predicted Fertilizer:", prediction[0])


Dataset Loaded Successfully!


,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,32,51,41,Red,Ground Nuts,7,3,19,14-35-14
1,35,58,35,Black,Cotton,4,14,16,Urea
2,27,55,43,Sandy,Sugarcane,28,0,17,20-20
3,33,56,56,Loamy,Ground Nuts,37,5,24,28-28
4,32,70,60,Red,Ground Nuts,4,6,9,14-35-14


KeyError: "['Temperature', 'Phosphorc'] not in index"

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import pickle

# Load dataset
data = pd.read_csv("Fertilizer Prediction.csv")
print("Dataset Loaded Successfully!")
display(data.head())

# Features (corrected column names)
X = data[['Temparature', 'Humidity', 'Moisture', 'Soil Type', 'Crop Type',
          'Nitrogen', 'Potassium', 'Phosphorous']]
y = data['Fertilizer Name']

# One-hot encode categorical columns
X = pd.get_dummies(X, columns=['Soil Type', 'Crop Type'])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Train the model
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Save model to pkl
with open("fertilizer_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("✅ fertilizer_model.pkl created successfully!")

# Test the model
sample = pd.DataFrame({
    'Temparature': [30],
    'Humidity': [60],
    'Moisture': [40],
    'Soil Type': ['Loamy'],
    'Crop Type': ['Sugarcane'],
    'Nitrogen': [20],
    'Potassium': [10],
    'Phosphorous': [15]
})

sample = pd.get_dummies(sample)
sample = sample.reindex(columns=X.columns, fill_value=0)

prediction = model.predict(sample)
print("Predicted Fertilizer:", prediction[0])


Dataset Loaded Successfully!


,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,32,51,41,Red,Ground Nuts,7,3,19,14-35-14
1,35,58,35,Black,Cotton,4,14,16,Urea
2,27,55,43,Sandy,Sugarcane,28,0,17,20-20
3,33,56,56,Loamy,Ground Nuts,37,5,24,28-28
4,32,70,60,Red,Ground Nuts,4,6,9,14-35-14


✅ fertilizer_model.pkl created successfully!
Predicted Fertilizer: 17-17-17


In [6]:
import pickle
import pandas as pd

# Load the trained model
with open("fertilizer_model.pkl", "rb") as f:
    model = pickle.load(f)

# Example input data (must match dataset column names exactly)
test_input = pd.DataFrame([{
    'Temparature': 32,
    'Humidity': 51,
    'Moisture': 41,
    'Soil Type': 'Red',
    'Crop Type': 'Ground Nuts',
    'Nitrogen': 7,
    'Potassium': 3,
    'Phosphorous': 19
}])

# One-hot encode the test data like in training
test_encoded = pd.get_dummies(test_input)

# Align with model training columns (fill missing with 0)
model_columns = model.feature_names_in_
test_encoded = test_encoded.reindex(columns=model_columns, fill_value=0)

# Predict
prediction = model.predict(test_encoded)
print("Predicted Fertilizer:", prediction[0])


Predicted Fertilizer: 14-35-14


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import pickle
import os

# ================== Load Dataset ==================
data_path = "../data/Fertilizer Prediction.csv"  # Adjust if running outside notebooks folder
data = pd.read_csv(data_path)
print("✅ Dataset Loaded Successfully!")
display(data.head())

# ================== Features & Target ==================
X = data[['Temparature', 'Humidity', 'Moisture', 'Soil Type', 'Crop Type',
          'Nitrogen', 'Potassium', 'Phosphorous']]
y = data['Fertilizer Name']

# One-hot encode categorical columns
X = pd.get_dummies(X, columns=['Soil Type', 'Crop Type'])

# ✅ Save column order for API usage
columns_path = "../API/fertilizer_columns.pkl"
with open(columns_path, "wb") as f:
    pickle.dump(X.columns.tolist(), f)
print(f"✅ fertilizer_columns.pkl saved to: {columns_path}")

# ================== Train-Test Split ==================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ================== Train the Model ==================
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
print("✅ Fertilizer model trained successfully!")

# ================== Save Model ==================
model_path = "../API/fertilizer_model.pkl"
with open(model_path, "wb") as f:
    pickle.dump(model, f)
print(f"✅ fertilizer_model.pkl saved to: {model_path}")

# ================== Test Model with a Sample ==================
sample = pd.DataFrame({
    'Temparature': [30],
    'Humidity': [60],
    'Moisture': [40],
    'Soil Type': ['Loamy'],
    'Crop Type': ['Sugarcane'],
    'Nitrogen': [20],
    'Potassium': [10],
    'Phosphorous': [15]
})

# One-hot encode & align with training columns
sample = pd.get_dummies(sample)
sample = sample.reindex(columns=X.columns, fill_value=0)

prediction = model.predict(sample)
print("💧 Predicted Fertilizer:", prediction[0])


✅ Dataset Loaded Successfully!


,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,32,51,41,Red,Ground Nuts,7,3,19,14-35-14
1,35,58,35,Black,Cotton,4,14,16,Urea
2,27,55,43,Sandy,Sugarcane,28,0,17,20-20
3,33,56,56,Loamy,Ground Nuts,37,5,24,28-28
4,32,70,60,Red,Ground Nuts,4,6,9,14-35-14


✅ fertilizer_columns.pkl saved to: ../API/fertilizer_columns.pkl
✅ Fertilizer model trained successfully!
✅ fertilizer_model.pkl saved to: ../API/fertilizer_model.pkl
💧 Predicted Fertilizer: 10-26-26


In [3]:
import requests

BASE_URL = "http://127.0.0.1:8000"

# Test 1: Check if server is running
print("🔍 Checking API health...")
resp = requests.get(BASE_URL)
print(resp.json())

# Test 2: Crop Prediction
print("\n🌱 Testing Crop Prediction...")
crop_payload = {
    "N": 90,
    "P": 40,
    "K": 40,
    "Temperature": 25,
    "Humidity": 80,
    "Ph": 6.5,
    "Rainfall": 200
}
resp = requests.post(f"{BASE_URL}/predict-crop", json=crop_payload)
print("Crop Prediction Response:", resp.json())

# Test 3: Fertilizer Prediction
print("\n💧 Testing Fertilizer Prediction...")
fert_payload = {
    "Temparature": 30,
    "Humidity": 60,
    "Moisture": 40,
    "Soil_Type": "Loamy",
    "Crop_Type": "Sugarcane",
    "Nitrogen": 20,
    "Potassium": 10,
    "Phosphorous": 15
}
resp = requests.post(f"{BASE_URL}/predict-fertilizer", json=fert_payload)
print("Fertilizer Prediction Response:", resp.json())


🔍 Checking API health...
{'message': '🌱 Crop & Fertilizer Prediction API is running!'}

🌱 Testing Crop Prediction...
Crop Prediction Response: {'detail': [{'type': 'missing', 'loc': ['body', 'ph'], 'msg': 'Field required', 'input': {'N': 90, 'P': 40, 'K': 40, 'Temperature': 25, 'Humidity': 80, 'Ph': 6.5, 'Rainfall': 200}}]}

💧 Testing Fertilizer Prediction...
Fertilizer Prediction Response: {'result': '10-26-26 is the best Fertilizer to use.'}


In [4]:
import requests

BASE_URL = "http://127.0.0.1:8000"

# Test 1: Check if server is running
print("🔍 Checking API health...")
resp = requests.get(BASE_URL)
print(resp.json())

# Test 2: Crop Prediction
print("\n🌱 Testing Crop Prediction...")
crop_payload = {
    "N": 90,
    "P": 40,
    "K": 40,
    "Temperature": 25,
    "Humidity": 80,
    "ph": 6.5,   # lowercase key so API matches the schema
    "Rainfall": 200
}
resp = requests.post(f"{BASE_URL}/predict-crop", json=crop_payload)
print("Crop Prediction Response:", resp.json())

# Test 3: Fertilizer Prediction
print("\n💧 Testing Fertilizer Prediction...")
fert_payload = {
    "Temparature": 30,
    "Humidity": 60,
    "Moisture": 40,
    "Soil_Type": "Loamy",
    "Crop_Type": "Sugarcane",
    "Nitrogen": 20,
    "Potassium": 10,
    "Phosphorous": 15
}
resp = requests.post(f"{BASE_URL}/predict-fertilizer", json=fert_payload)
print("Fertilizer Prediction Response:", resp.json())


🔍 Checking API health...
{'message': '🌱 Crop & Fertilizer Prediction API is running!'}

🌱 Testing Crop Prediction...
Crop Prediction Response: {'error': "'CROP' object has no attribute 'Ph'"}

💧 Testing Fertilizer Prediction...
Fertilizer Prediction Response: {'result': '10-26-26 is the best Fertilizer to use.'}


In [5]:
import requests

BASE_URL = "http://127.0.0.1:8000"

# Test 1: Check if server is running
print("🔍 Checking API health...")
resp = requests.get(BASE_URL)
print(resp.json())

# Test 2: Crop Prediction
print("\n🌱 Testing Crop Prediction...")
crop_payload = {
    "N": 90,
    "P": 40,
    "K": 40,
    "Temperature": 25,
    "Humidity": 80,
    "Ph": 6.5,  # JSON key still 'Ph', backend maps to .ph internally
    "Rainfall": 200
}
resp = requests.post(f"{BASE_URL}/predict-crop", json=crop_payload)
print("Crop Prediction Response:", resp.json())

# Test 3: Fertilizer Prediction
print("\n💧 Testing Fertilizer Prediction...")
fert_payload = {
    "Temparature": 30,  # Keep spelling same as dataset
    "Humidity": 60,
    "Moisture": 40,
    "Soil_Type": "Loamy",
    "Crop_Type": "Sugarcane",
    "Nitrogen": 20,
    "Potassium": 10,
    "Phosphorous": 15
}
resp = requests.post(f"{BASE_URL}/predict-fertilizer", json=fert_payload)
print("Fertilizer Prediction Response:", resp.json())


🔍 Checking API health...
{'message': '🌱 Crop & Fertilizer Prediction API is running!'}

🌱 Testing Crop Prediction...
Crop Prediction Response: {'result': 'Recommended Crop: Unknown Crop'}

💧 Testing Fertilizer Prediction...
Fertilizer Prediction Response: {'result': '10-26-26 is the best Fertilizer to use.'}


In [6]:
import requests

BASE_URL = "http://127.0.0.1:8000"

# Test 1: API health check
print("🔍 Checking API health...")
resp = requests.get(BASE_URL)
print(resp.json())

# Test 2: Crop Prediction
# These values are chosen to match the typical ranges from your training dataset
print("\n🌱 Testing Crop Prediction...")
crop_payload = {
    "N": 90,          # Nitrogen
    "P": 40,          # Phosphorous
    "K": 40,          # Potassium
    "Temperature": 25,  # Celsius
    "Humidity": 80,     # %
    "ph": 6.5,          # Soil pH - lowercase to match schema
    "Rainfall": 200     # mm
}
resp = requests.post(f"{BASE_URL}/predict-crop", json=crop_payload)
print("Crop Prediction Response:", resp.json())

# Test 3: Fertilizer Prediction
print("\n💧 Testing Fertilizer Prediction...")
fert_payload = {
    "Temparature": 30,   # Keep spelling same as dataset
    "Humidity": 60,
    "Moisture": 40,
    "Soil_Type": "Loamy",
    "Crop_Type": "Sugarcane",
    "Nitrogen": 20,
    "Potassium": 10,
    "Phosphorous": 15
}
resp = requests.post(f"{BASE_URL}/predict-fertilizer", json=fert_payload)
print("Fertilizer Prediction Response:", resp.json())


🔍 Checking API health...
{'message': '🌱 Crop & Fertilizer Prediction API is running!'}

🌱 Testing Crop Prediction...
Crop Prediction Response: {'detail': [{'type': 'missing', 'loc': ['body', 'Ph'], 'msg': 'Field required', 'input': {'N': 90, 'P': 40, 'K': 40, 'Temperature': 25, 'Humidity': 80, 'ph': 6.5, 'Rainfall': 200}}]}

💧 Testing Fertilizer Prediction...
Fertilizer Prediction Response: {'result': '10-26-26 is the best Fertilizer to use.'}


In [7]:
import requests

BASE_URL = "http://127.0.0.1:8000"

# Test 1: Health check
print("🔍 Checking API health...")
resp = requests.get(BASE_URL)
print(resp.json())

# Test 2: Crop Prediction
print("\n🌱 Testing Crop Prediction...")
crop_payload = {
    "N": 90,
    "P": 40,
    "K": 40,
    "Temperature": 25,
    "Humidity": 80,
    "ph": 6.5,  # lowercase
    "Rainfall": 200
}
resp = requests.post(f"{BASE_URL}/predict-crop", json=crop_payload)
print("Crop Prediction Response:", resp.json())

# Test 3: Fertilizer Prediction
print("\n💧 Testing Fertilizer Prediction...")
fert_payload = {
    "Temparature": 30,
    "Humidity": 60,
    "Moisture": 40,
    "Soil_Type": "Loamy",
    "Crop_Type": "Sugarcane",
    "Nitrogen": 20,
    "Potassium": 10,
    "Phosphorous": 15
}
resp = requests.post(f"{BASE_URL}/predict-fertilizer", json=fert_payload)
print("Fertilizer Prediction Response:", resp.json())


🔍 Checking API health...
{'message': '🌱 Crop & Fertilizer Prediction API is running!'}

🌱 Testing Crop Prediction...
Crop Prediction Response: {'result': 'Recommended Crop: Unknown Crop'}

💧 Testing Fertilizer Prediction...
Fertilizer Prediction Response: {'result': '10-26-26 is the best Fertilizer to use.'}


In [8]:
import requests

BASE_URL = "http://127.0.0.1:8000"

# 🔍 Check API health
print("🔍 Checking API health...")
print(requests.get(BASE_URL).json())

# 💧 Test Fertilizer Prediction
fert_payload = {
    "Temparature": 26,
    "Humidity": 52,
    "Moisture": 38,
    "Soil_Type": "Sandy",
    "Crop_Type": "Maize",
    "Nitrogen": 37,
    "Potassium": 0,
    "Phosphorous": 0
}

print("\n💧 Testing Fertilizer Prediction...")
res = requests.post(f"{BASE_URL}/predict-fertilizer", json=fert_payload)
print("Fertilizer Prediction Response:", res.json())


🔍 Checking API health...
{'message': '🌱 Crop & Fertilizer Prediction API is running!'}

💧 Testing Fertilizer Prediction...
Fertilizer Prediction Response: {'result': '10-26-26 is the best Fertilizer to use.'}


In [9]:
# 💧 Fertilizer Prediction Test Script

import requests

BASE_URL = "http://127.0.0.1:8000"

# Test 1: API Health Check
print("🔍 Checking API health...")
resp = requests.get(BASE_URL)
print(resp.json())

# Test 2: Fertilizer Prediction
print("\n💧 Testing Fertilizer Prediction...")
fert_payload = {
    "Temparature": 30,
    "Humidity": 60,
    "Moisture": 40,
    "Soil_Type": "Loamy",
    "Crop_Type": "Sugarcane",
    "Nitrogen": 20,
    "Potassium": 10,
    "Phosphorous": 15
}

resp = requests.post(f"{BASE_URL}/predict-fertilizer", json=fert_payload)
print("Fertilizer Prediction Response:", resp.json())


🔍 Checking API health...
{'message': '🌱 Crop & Fertilizer Prediction API is running!'}

💧 Testing Fertilizer Prediction...
Fertilizer Prediction Response: {'result': '10-26-26 is the best Fertilizer to use.'}


In [10]:
import requests

# Your API URL
BASE_URL = "http://127.0.0.1:8000"

# 1️⃣ Check API health
print("🔍 Checking API health...")
health = requests.get(f"{BASE_URL}/").json()
print(health)

# 2️⃣ Test Fertilizer Prediction
print("\n💧 Testing Fertilizer Prediction...")

fertilizer_payload = {
    "Temparature": 32,        # Keep spelling same as training data
    "Humidity": 51,
    "Moisture": 41,
    "Soil_Type": "Red",
    "Crop_Type": "Ground Nuts",
    "Nitrogen": 7,
    "Potassium": 3,
    "Phosphorous": 19
}

fert_response = requests.post(
    f"{BASE_URL}/predict-fertilizer",
    json=fertilizer_payload
).json()

print("Fertilizer Prediction Response:", fert_response)


🔍 Checking API health...
{'message': '🌱 Crop & Fertilizer Prediction API is running!'}

💧 Testing Fertilizer Prediction...
Fertilizer Prediction Response: {'result': '14-35-14 is the best Fertilizer to use.'}


In [1]:
# 📌 Fertilizer Prediction Training

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import pickle
import os
from sklearn.metrics import accuracy_score

# ================== Load Dataset ==================
data_path = "../data/Fertilizer Prediction.csv"
data = pd.read_csv(data_path)
print("✅ Dataset Loaded Successfully!")

# ================== Features & Target ==================
X = data.drop('Fertilizer Name', axis=1)
y = data['Fertilizer Name']

# One-hot encode categorical columns
X = pd.get_dummies(X, columns=['Soil Type', 'Crop Type'])

# ================== Save Columns and Model ==================
# This is the most crucial part for your API.
columns_path = "../API/fertilizer_columns.pkl"
with open(columns_path, "wb") as f:
    pickle.dump(X.columns.tolist(), f)
print(f"\n✅ fertilizer_columns.pkl saved to: {columns_path}")

model_path = "../API/fertilizer_model.pkl"
with open(model_path, "wb") as f:
    pickle.dump(model, f)
print(f"✅ fertilizer_model.pkl saved to: {model_path}")

# ================== Train-Test Split & Evaluation ==================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"\n✅ Fertilizer Model Accuracy: {accuracy_score(y_test, y_pred):.2f}")

# ================== Print Final Column List ==================
print("\n📋 Final One-Hot Encoded Columns:")
print(X.columns.tolist())

✅ Dataset Loaded Successfully!

✅ fertilizer_columns.pkl saved to: ../API/fertilizer_columns.pkl


NameError: name 'model' is not defined

In [1]:
# 📌 Fertilizer Model Training

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import pickle

# 1️⃣ Load dataset
data = pd.read_csv("../data/Fertilizer Prediction.csv")
print("✅ Dataset Loaded Successfully!")
display(data.head())

# 2️⃣ Features and Target
X = data[['Temparature', 'Humidity', 'Moisture', 'Soil Type', 'Crop Type',
          'Nitrogen', 'Potassium', 'Phosphorous']]
y = data['Fertilizer Name']

# 3️⃣ One-hot encode categorical columns
X_encoded = pd.get_dummies(X, columns=['Soil Type', 'Crop Type'])

# 4️⃣ Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)

# 5️⃣ Train the model
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
print("✅ Model trained successfully!")

# 6️⃣ Save model to API folder
with open("../API/fertilizer_model.pkl", "wb") as f:
    pickle.dump(model, f)
print("🎉 fertilizer_model.pkl saved successfully!")


✅ Dataset Loaded Successfully!


,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,32,51,41,Red,Ground Nuts,7,3,19,14-35-14
1,35,58,35,Black,Cotton,4,14,16,Urea
2,27,55,43,Sandy,Sugarcane,28,0,17,20-20
3,33,56,56,Loamy,Ground Nuts,37,5,24,28-28
4,32,70,60,Red,Ground Nuts,4,6,9,14-35-14


✅ Model trained successfully!
🎉 fertilizer_model.pkl saved successfully!


In [2]:
import pandas as pd
import pickle

# 1️⃣ Load dataset
data_path = "../data/Fertilizer Prediction.csv"
data = pd.read_csv(data_path)

# 2️⃣ Select features (excluding target)
X = data.drop('Fertilizer Name', axis=1)

# 3️⃣ One-hot encode categorical columns
X_encoded = pd.get_dummies(X, columns=['Soil Type', 'Crop Type'])

# 4️⃣ Save column names
columns_path = "../API/fertilizer_columns.pkl"
with open(columns_path, "wb") as f:
    pickle.dump(X_encoded.columns.tolist(), f)

print("📦 fertilizer_columns.pkl saved successfully!")
print("📋 Columns:", X_encoded.columns.tolist())


📦 fertilizer_columns.pkl saved successfully!
📋 Columns: ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous', 'Soil Type_Black', 'Soil Type_Clayey', 'Soil Type_Loamy', 'Soil Type_Red', 'Soil Type_Sandy', 'Crop Type_Barley', 'Crop Type_Cotton', 'Crop Type_Ground Nuts', 'Crop Type_Maize', 'Crop Type_Millets', 'Crop Type_Oil seeds', 'Crop Type_Paddy', 'Crop Type_Pulses', 'Crop Type_Sugarcane', 'Crop Type_Tobacco', 'Crop Type_Wheat']
